# Predicting Peak Usage on a Minecraft Research Server

## Introduction

A research group in Computer Science at UBC which is led by Frank Wood is studying how people play Minecraft. They have set up a special Minecraft server that tracks when players join and leave. Since running the server requires resources such as server space and licenses, they want to ensure they have enough resources when the most people are playing.

In this project, we will try to find patterns about when players are most active. If we can find which times of the day are the busiest, the research team can better plan when to prepare more server resources. In particular, we will address the question: Can we predict the level of activity on the Minecraft server based on the day of the week and time of day?

There are two available files: "player.csv" and "sessions.csv". The file we are going to use is called "sessions.csv". It has 1535 observations with 5 variables. The following table is a basic summary:

| Variable Name         | Type   | Description                                          |
| --------------------- | ------ | ---------------------------------------------------- |
| `hashedEmail`         | chr    | A unique ID for each player |
| `start_time`          | chr | The start date and time of each player             |
| `end_time`            | chr | The start date and time of each player               |
| `original_start_time` | dbl  | Start time in milliseconds       |
| `original_end_time`   | dbl  | End time in milliseconds         |


Since we are interested in demand forecasting, we will use the "start_time" column and the "end_time" column.

In [4]:
library(tidyverse)
library(repr)
library(readxl)
source("cleanup.R")
options(repr.matrix.max.rows = 6)

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'cleanup.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [9]:
session <- read_csv("sessions.csv") |>
            select(start_time, end_time)
session

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


start_time,end_time
<chr>,<chr>
30/06/2024 18:12,30/06/2024 18:24
17/06/2024 23:33,17/06/2024 23:46
25/07/2024 17:34,25/07/2024 17:57
25/07/2024 03:22,25/07/2024 03:58
25/05/2024 16:01,25/05/2024 16:12
23/06/2024 15:08,23/06/2024 17:10
15/04/2024 07:12,15/04/2024 07:21
21/09/2024 02:13,21/09/2024 02:30
21/06/2024 02:31,21/06/2024 02:49
